In [9]:
# Created on Thu Dec 20 13:23:15 2018
#
# @author: espenfb

In [10]:
import os
import sys
from pathlib import Path

filepath = os.path.abspath('../')
sys.path.append(str(Path(filepath) / "src"))

import detInvModel as dim
import pandas as pd
pd.set_option("plotting.backend","plotly")
import savedRes as sr

In [11]:
time_data = {
    'start_date': pd.Timestamp(year=2019, month=1, day=1),
    'end_date': pd.Timestamp(year=2020, month=1, day=1),
    'ref_date': pd.Timestamp(year=2019, month=1, day=1)}
dirs = {
    'data_dir': "data\\",
    'ctrl_data_file': 'ctrl_data.csv',
    'res_dir': 'results_test\\'}

In [12]:
obj = dim.deterministicModel(time_data, dirs)

0 load Series/load_series.csv
data\Series/load_series.csv
1 wind Series/pot_wind_profiles.csv
data\Series/pot_wind_profiles.csv
2 inst_wind Series/inst_wind_profiles.csv
data\Series/inst_wind_profiles.csv
4 solar Series/solar_profiles.csv
data\Series/solar_profiles.csv
5 offshore_wind Series/offshore_wind_series.csv
data\Series/offshore_wind_series.csv
6 inflow Series/inflow.csv
data\Series/inflow.csv
7 inflow_ureg Series/inflow_ureg.csv
data\Series/inflow_ureg.csv
8 import_price Series/import_price_utc.csv
data\Series/import_price_utc.csv
9 export_price Series/export_price_utc.csv
data\Series/export_price_utc.csv


In [13]:
obj.buildModel()

Building deterministic investment model...
{'Biomass': [], 'CC Gas': [], 'CT Gas': [], 'CCS Gas': [], 'CT H2': [], 'CC H2': [], 'Coal': [], 'CCS Coal': [], 'Nuclear': [], 'PEMFC': ['PEMFC01', 'PEMFC02', 'PEMFC03', 'PEMFC04', 'PEMFC05', 'PEMFC06', 'PEMFC07', 'PEMFC08', 'PEMFC13'], 'PEMEL': ['PEMEL01', 'PEMEL02', 'PEMEL03', 'PEMEL04', 'PEMEL05', 'PEMEL06', 'PEMEL07', 'PEMEL08', 'PEMEL13'], 'SMR': [], 'SMR CCS': []}
Creating LP problem instance...


In [14]:
import detInvData as did
import copy
import numpy as np
obj.detDataInstance = did.detData(obj)
di = obj.detDataInstance["detData"]

{'Biomass': [], 'CC Gas': [], 'CT Gas': [], 'CCS Gas': [], 'CT H2': [], 'CC H2': [], 'Coal': [], 'CCS Coal': [], 'Nuclear': [], 'PEMFC': ['PEMFC01', 'PEMFC02', 'PEMFC03', 'PEMFC04', 'PEMFC05', 'PEMFC06', 'PEMFC07', 'PEMFC08', 'PEMFC13'], 'PEMEL': ['PEMEL01', 'PEMEL02', 'PEMEL03', 'PEMEL04', 'PEMEL05', 'PEMEL06', 'PEMEL07', 'PEMEL08', 'PEMEL13'], 'SMR': [], 'SMR CCS': []}


In [15]:
solar_series = copy.copy(obj.data.solar_series)
solar_series = solar_series[solar_series.index.isin(obj.time)]
solar_series.index = pd.Index(np.arange(len(solar_series.index)))
solar_series.rename(columns={str(i): obj.type2prefix['Solar'] + '%.2d' % int(i)
                             for i in obj.plant_buses_type["Solar"]},
                    level=0, inplace=True)
solar_series[solar_series < 0] = 0.0
include_plants = di["SOLAR_POWER_PLANTS"][None]
solar_series = solar_series[include_plants]
di['Renewable_profile'] = solar_series.round(4).stack(level=0).to_dict()


In [16]:
obj.instance

In [17]:
obj.solve()

Solving deterministic operation model...


In [18]:
obj.processResults()

Prosessing results from deteministic model...
    (deprecated in 6.0) (called from
    c:\Users\espenb\Models\HEIM\src\detInvModel.py:111)
    (deprecated in 6.0) (called from
    c:\Users\espenb\Models\HEIM\src\detInvModel.py:111)
    (deprecated in 6.0) (called from
    c:\Users\espenb\Models\HEIM\src\detInvModel.py:111)


In [19]:
obj.saveRes(dirs['res_dir'])

In [ ]:
res = sr.savedRes(dirs['res_dir'], data=obj.data)